# ASSIGNMENT 1: AUTOMATED MARKET OPENING SCHEDULER DURING COVID


> Goal: The goal of this assignment is to take a complex new problem and formulate and solve it as search. Formulation as search is an integral skill of AI that will come in handy whenever you are faced with a new problem. Heuristic search will allow you to find optimal solutions. Local search may not find the optimal solution, but is usually able to find good solutions for really large problems.


## Scenario: Optimization of Market Opening Schedule.
A city has **n** types of shops. 

The government wants to create an opening schedule for the markets ensuring the safety of maximum people. Due to the current COVID situation the government wants the people to make minimum movement out of their houses. They have approached you to take your help in order to organize the opening of shops in a best possible schedule. You need to use the power of AI and write a generalized search algorithm to find the best possible schedule. 

The city has **m** market places which can be opened parallely. In a market place during each time slot the government is planning to open **k** types of shops. And in a day there are a total of **T** time slots available. We can assume that `n = T.m.k`. 

For example, in figure below `m = 2`, `T = 3` and `k = 4`

||||
|:-|:-|:-|
|Type: 1,2,3,4|Type: 5,6,7,8|Type: 9,10,11,12|
|Type: 13,14,15,16|Type: 17,18,19,20|Type: 21,22,23,24|


We first define the characteristics of a good schedule. For any good schedule people should make minimum movement and most of the people should feel no conflict about which market they should go for purchasing.

That is: 
- All types of shops opening in one time slot in the same market should sell related items (items generally bought together).
- All types of shops opening in parallel markets should be as far away as possible to avoid people’s movement to all of the markets (selling items that are generally not bought together).

To operationalize this intuition let us assume we are given a function representing the distance between two types/categories: **`d(t1, t2)`**, such that **d** is between 0 and 1. We can similarly define a similarity between two, **`s(t1, t2) = 1 - d(t1, t2)`**. 

Now we can define the goodness of a schedule as follows:
- `Sum(similarities of all pairs within a single time slot in the same market) + C * Sum(distances of all pairs within a single time slot in the parallel market)`.

In our example, the goodness will be computed as,

> ```
> G(Schedule) = s(1,2) + s(1,3) + s(1,4) + s(2,3) + s(2,4) + s(3,4) + 
>               s(5,6) + s(5,7) + s(5,8) + s(6,7) + s(6,8) + s(7,8) +
>         ……. + s(13,14) + …. + s(21,22) + ….. + 
>    + C x [d(1,13) + d(1,14)… d(2,13) + d(2,14) + … + d(5,17) + d(5,18) + …]
> ```

The constant **C** trades off the importance of semantic coherence of one market versus reducing conflict across parallel markets.

**Your goal is to find a schedule with the maximum goodness.**


Input:

Line 1: **k**: total types of shops opening in one time slot in one market

Line 2: **m**: number of parallel markets

Line 3: **T**: number of time slots

Line 4: **C**: trade-off constant

Starting on the fifth line we have a space separated list of distances between a type of shop and rest others. Note that `d(x,y) = d(y,x)`. Also, all `d(x,x) = 0`.


Important Instructions:
- You may work in teams of maximum three or by yourself. If you are short of partners, our recommendation is that this assignment is quite straightforward and a partner should not be required.
- You cannot use built-in libraries/implementations for search or scheduling algorithms.
- Please do not search the Web for solutions to the problem. Your submission will be checked for plagiarism with the codes available on Web as well as the codes submitted by other teams. Any team found guilty will be awarded a suitable penalty as per IIT rules.
- Your code will be automatically evaluated. You get a zero if your output is not automatically parsable.
- You are allowed to use any of the two programming languages: C++, Python.


### Sample Input
```
2
2
1
1
0 0.4 0.8 1
0.4 0 0.6 0.7
0.8 0.6 0 0.3
1 0.7 0.3 0
```
### Output
Your algorithm should return the max-goodness schedule.

#### Output Format:
Space separated list of shop ids (i.e, shop’s type ids), where time slots are separated by bars and parallel markets are separated by line.
For the above problem the optimal solution is t1 and t2 in one market; and t3 and t4 in the other market. It will be represented as: 
```
1 2
3 4
```
Other equivalent ways to represent this same solution:
```
4 3
2 1
```
OR
```
2 1
3 4
```
etc. All are valid and have the total goodness of 4.4 (Verify).

Another sample input is provided along with the assignment representing similar easy problems.

We recommend you to experiment with other problems as well.


# Utility functions for colab notebook

## Ray for parallelism
Install and setup the Ray library.
https://github.com/ray-project/tutorial

In [3]:
# Install Ray for parallelism
print('NOTE: Intentionally crashing session to use the newly installed library.\n')

!pip uninstall -y pyarrow
!pip install ray[debug]==0.7.5
!pip install bs4

try:
    import ray
except:
    # A hack to force the runtime to restart, needed to include the above dependencies.
    import os
    os._exit(0)

NOTE: Intentionally crashing session to use the newly installed library.

     |████████████████████████████████| 74.9 MB 21 kB/s 
     |████████████████████████████████| 269 kB 2.3 MB/s 
     |████████████████████████████████| 72 kB 7.7 kB/s 
     |████████████████████████████████| 14.5 MB 13 kB/s 
     |████████████████████████████████| 56 kB 59 kB/s 
     |████████████████████████████████| 82 kB 13 kB/s 
     |████████████████████████████████| 272 kB 59 kB/s 
     |████████████████████████████████| 1.3 MB 447 kB/s 
     |████████████████████████████████| 460 kB 291 kB/s 
     |████████████████████████████████| 2.9 MB 159 kB/s 
     |████████████████████████████████| 48 kB 122 kB/s 
     |████████████████████████████████| 106 kB 1.7 MB/s 
     |████████████████████████████████| 99 kB 332 kB/s 
     |████████████████████████████████| 67 kB 64 kB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=471240 sha256=005b4fadd6426acf8d66088dda00ee61d93201fb7

## I/O Helper Functions

### set_input(inp_str)
This takes a string which will be used to read lines for the input function.

### input()
This is a proxy to use the given string instead of asking the user for input.

In [4]:
# =======================================================
#       Fake input function for colab notebook
# =======================================================
def set_input(inp_str):
    global __input
    __input = [x.strip() for x in inp_str.split('\n')]
    __input = [x for x in __input if x!=""]
    global __fake_input_ctr
    __fake_input_ctr = -1
def input() -> str:
    global __fake_input_ctr
    __fake_input_ctr += 1
    return __input[__fake_input_ctr]
# =======================================================

## Test cases
These are test cases used to test the program.

A test case is just an input string.

`TestCases` is a list of test cases.

In [5]:
TestCases = [
                  """
                  2
                  2
                  1
                  1
                  0 0.4 0.8 1
                  0.4 0 0.6 0.7
                  0.8 0.6 0 0.3
                  1 0.7 0.3 0
                  """,
                  """
                  2
                  2
                  3
                  1
                  0 0.0 0.8 0.9 0.2 0.1 1.0 0.8 0.2 0.3 0.8 0.8
                  0.0 0 0.8 1.0 0.2 0.1 0.8 0.9 0.2 0.2 0.8 1.0
                  0.8 0.8 0 0.1 0.8 0.9 0.2 0.2 0.7 0.9 0.0 0.2
                  0.9 1.0 0.1 0 0.7 0.9 0.0 0.2 0.9 0.9 0.1 0.1
                  0.2 0.2 0.8 0.7 0 0.1 0.8 1.0 0.3 0.2 0.9 0.7
                  0.1 0.1 0.9 0.9 0.1 0 0.8 0.9 0.1 0.2 0.8 0.9
                  1.0 0.8 0.2 0.0 0.8 0.8 0 0.0 0.9 0.8 0.1 0.0
                  0.8 0.9 0.2 0.2 1.0 0.9 0.0 0 0.8 0.9 0.2 0.0
                  0.2 0.2 0.7 0.9 0.3 0.1 0.9 0.8 0 0.2 0.8 0.8
                  0.3 0.2 0.9 0.9 0.2 0.2 0.8 0.9 0.2 0 0.8 0.9
                  0.8 0.8 0.0 0.1 0.9 0.8 0.1 0.2 0.8 0.8 0 0.3
                  0.8 1.0 0.2 0.1 0.7 0.9 0.0 0.0 0.8 0.9 0.3 0
                  """,
]

In [6]:
# Generate test cases
def generate_test_case(K,M,T,C) -> str:
    N = K*M*T
    r = f"{K}\n{M}\n{T}\n{C}\n"
    import numpy as np
    dist = np.random.rand(N,N)
    dist = np.around(np.maximum(dist, dist.transpose()), decimals = 1)
    for i in dist:
        r += " ".join(map(str,i)) + "\n"
    return r

# generate test case for N=100
TestCases.append(generate_test_case(10,5,2,2.4))

# Typedefs and Data Structures

## Context
This stores the variables `K`, `M`, `T`, `C` and `N` as derived from the input.

This DS is immutable, i.e. once created, data inside it cannot be changed.

In [7]:
# Typings
from dataclasses import dataclass, field
# Context stores all the input data
@dataclass(frozen=True)
class Context:
    K: int
    M: int
    T: int
    C: float
    N: int = field(init=False)
    def __post_init__(self):
        # set immutable value of N
        # workaround https://stackoverflow.com/questions/53756788/how-to-set-the-value-of-dataclass-field-in-post-init-when-frozen-true
        object.__setattr__(self, 'N', self.K * self.T * self.M)

## SymmetricMatrix
A class to efficiently store symmetric matrices.

`m[1,2]` = element at `row=1` and `col=2`.

In [35]:
# Typings
from typing import Tuple
# Distance matrix data structure
# A symmetric square matrix of size NxN
class SymmetricMatrix:
    def __init__(self, size: int):
        # allocate space for lower half
        self.matrix = [ [None]*(i+1) for i in range(size) ]
        self.size = size
        # allocate space for cache
        self._cache_sum = None
    # get matrix element
    def __getitem__(self, pos: Tuple[int, int]):
        row, col = pos
        # ensure row >= col
        if(row < col):
            row, col = col, row
        return self.matrix[row][col]
    # set matrix element
    def __setitem__(self, pos: Tuple[int, int], newvalue):
        row, col = pos
        # ensure row >= col
        if(row < col):
            row, col = col, row
        self.matrix[row][col] = newvalue
        # invalidate sum cache
        self._cache_sum = None
    # define str function
    def __str__(self):
        s = f"SymmetricMatrix(size={self.size}) ["
        for i in range(self.size):
            s += "\n\t" + " ".join(map(str,self.matrix[i]))
        s += "\n]"
        return s
    # returns the sum of all values
    def sum(self):
        if(self._cache_sum == None):
            # calc. sum
            self._cache_sum = v = 0
            for i in self.matrix:
                v += sum(i)
                self._cache_sum -= i[-1]
            self._cache_sum = v*2
        return self._cache_sum
    # returns average of all values
    def average(self):
        return self.sum() / self.size**2

## Schedule
A class that represents a schedule as a table with `T` rows and `m` columns with each cell containing `k` elements.

In [175]:
from typing import List, Callable
import random
class Schedule:
    # Represents a single cell
    class Cell:
        # a func. that takes the cell instance and two int, shop removed and shop added
        CelllUpdateListener = Callable[[Schedule.Cell, int, int], None]
        def __init__(self, context: Context, distances: SymmetricMatrix):
            self.context = context
            self.distances = distances
            # allocate shops
            self.shops = [None] * context.K
            # allocate cache
            self._invalidate_cache()
            # alocate listeners
            self._listeners = dict()
        # invalidate all cache
        def _invalidate_cache(self):
            self._cache_G = None
        # replace cell with contents of list
        def _fromList(self, lst: List[int]):
            self.shops = lst
            self._invalidate_cache()
        # getter for G value of just this cell
        @property
        def G(self) -> float:
            if(self._cache_G == None):
                # calc G
                self._cache_G = 0
                for i in range(len(self.shops)):
                    for j in range(i+1, len(self.shops)):
                        self._cache_G += 1 - self.distances[self.shops[i]-1, self.shops[j]-1]
            return self._cache_G
        # returns D between this cell and the other
        def calcD(self, other: Schedule.Cell) -> float:
            r = 0
            for s in self.shops:
                for l in other.shops:
                    r += self.distances[s-1,l-1]
            return r
        # returns the expectedG of a cell with k shops
        @staticmethod
        def expectedG(k: int, avgDist: float) -> float:
            # nC2 * avgDist
            K = self.context.K
            return (K*(K-1)/2) * avgDist
        # __getitem__ function overloads the [] operator
        # [x] will fetch the shop x in cell
        def __getitem__(self, pos):
            if(type(pos) == tuple):
                if(len(pos) == 1):
                    return self.shops[pos[0]]
            else:
                return self.shops[pos]
        def __setitem__(self, pos, newval):
            oldval = self.shops[pos]
            # replace shop with new val and invalidate cache
            self.shops[pos] = newval
            self._invalidate_cache()
            # fire cahnge listeners
            for id, listener in self._listeners.items():
                listener(self, oldval, newval)
        # to string
        def __str__(self):
            return " ".join([str(s) for s in self.shops])
        # register change listeners
        def _registerChangeListener(self, listener: CelllUpdateListener, register_id):
            self._listeners[register_id] = listener
        def _unregisterChangeListener(self, register_id):
            self._listeners.pop(register_id, None)
    
    # Represents a single timeslot
    class Timeslot:
        def __init__(self, context: Context, distances: SymmetricMatrix):
            self.context = context
            self.distances = distances
            # allocate cells
            self.cells = [ Schedule.Cell(context, distances) for i in range(context.M) ]
            # register change listeners
            for cell in self.cells:
                cell._registerChangeListener(self.onCellChange, self)
            # allocate cache
            self._cache_D = [None] * context.M
            self._invalidate_cache()
        # called when a cell contents change
        def onCellChange(self, cell: Schedule.Cell, removedShop: int, addedShop: int):
            # update D cache if D cache is valid
            if(self._cache_D_valid):
                updated_idx = -1
                total_delta = 0
                for i in range(len(self.cells)):
                    if(self.cells[i]==cell):
                        updated_idx = i
                        continue
                    toRem = sum(map(lambda s: self.distances[s-1,removedShop-1], self.cells[i].shops))
                    toAdd = sum(map(lambda s: self.distances[s-1,addedShop-1], self.cells[i].shops))
                    self._cache_D[i] += -toRem + toAdd
                    total_delta += -toRem + toAdd
                self._cache_D[updated_idx] += total_delta
            # invalidate G cache
            self._cache_G = None
        # invalidate all cache
        def _invalidate_cache(self):
            self._cache_D_valid = False
            self._cache_G = None
        # replace timeslot with contents of list
        def _fromList(self, lst: List[int]):
            # apply list to cells
            for i in range(len(self.cells)):
                self.cells[i]._fromList(lst[i*self.context.K : (i+1)*self.context.K])
            self._invalidate_cache()
        # getter for G value of this timeslot
        @property
        def G(self):
            if(self._cache_G == None):
                if(self._cache_D_valid == False):
                    self._buildDCache()
                self._cache_G = sum(map(lambda cell: cell.G, self.cells)) + (sum(self._cache_D) / 2)*self.context.C
            return self._cache_G
        # getter for D value of this timeslot
        @property
        def D(self):
            if(self._cache_D_valid == False):
                self._buildDCache()
            return sum(self._cache_D) / 2
        # returns the total D value of a cell in this timeslot
        def getCellD(self, cell_idx: int):
            # build cache if needed
            if(self._cache_D_valid == False):
                self._buildDCache()
            return self._cache_D[cell_idx]
        # returns the expectedG of a timeslot with m markets and k shops per cell
        @staticmethod
        def expectedG(m: int, k: int, c: int, avgDist: float) -> float:
            return self.expectedS(m,k,avgDist) + c*self.expectedD(m,k,avgDist)
        # returns the expectedS of a timeslot with m markets and k shops per cell
        @staticmethod
        def expectedS(m: int, k: int, avgDist: float) -> float:
            return m*Schedule.Cell.expectedG(k,avgDist)
        # returns the expectedD of a timeslot with m markets and k shops per cell
        @staticmethod
        def expectedD(m: int, k: int, avgDist: float) -> float:
            return self.expectedCellD(m,k,avgDist)*m/2
        # returns the expectedD of a cell in a timeslot with m markets and k shops per cell
        # the total D value of a timeslot is half of (this value * M)
        @staticmethod
        def expectedCellD(k: int, avgDist: float) -> float:
            return avgDist*(k**2)/2
        # builds D cache
        def _buildDCache(self):
            M = self.context.M
            self._cache_D = [0] * M
            for i in range(M):
                for j in range(i+1, M):
                    D = self.cells[i].calcD(self.cells[j])
                    self._cache_D[i] += D
                    self._cache_D[j] += D
            self._cache_D_valid = True
        # __getitem__ function overloads the [] operator
        # [y,z] will fetch y cell -> shop at position z in the cell
        # [x] will fetch the entire x cell
        def __getitem__(self, pos):
            # magic
            if (type(pos) == tuple):
                if(len(pos) == 1):
                    return self.cells[pos[0]]
                return self.cells[pos[0]][pos[1:]]
            else:
                return self.cells[pos]
        def __setitem__(self, pos, newval: Schedule.Cell):
            M = self.context.M
            oldval = self.cells[pos]
            self.cells[pos] = newval
            # reset pos cache D
            self._cache_D[pos] = 0
            # update D cache if it is valid
            if(self._cache_D_valid):
                for i in range(M):
                    if(i==pos):
                        continue
                    # add new cell D value
                    delta = self.cells[i].calcD(newval)
                    self._cache_D[pos] += delta
                    # remove prev cell D value
                    delta -= self.cells[i].calcD(oldval)
                    self._cache_D[i] += delta
            # invalidate G
            self._cache_G = None
        # to string
        def __str__(self):
            return "\n".join(
                [
                    f"| {str(c)} |" for c in self.cells
                ]
            )

    def __init__(self, context: Context, distances: SymmetricMatrix):
        self.context = context
        self.distances = distances
        # allocate timeslots
        self.timeslots = [ Schedule.Timeslot(context, distances) for i in range(context.T) ]
    # replace timeslot with contents of list
    def _fromList(self, lst: List[int]):
        # apply list to cells
        sz = self.context.K * self.context.M
        for i in range(len(self.timeslots)):
            self.timeslots[i]._fromList(lst[i*sz : (i+1)*sz])
    # This randomises the schedule
    def randomize(self, seed = random.seed()):
        serial_sch = list(range(1,self.context.N+1))
        # shuffle schedule
        random.Random(seed).shuffle(serial_sch)
        # set schedule contents to contents of serial_sch
        self._fromList(serial_sch)
    # __getitem__ function overloads the [] operator
    # [x,y,z] will fetch x timeslot -> y market -> shop at position z in the cell
    # [x] will fetch the entire x timeslot
    def __getitem__(self, pos):
        # magic
        if (type(pos) == tuple):
            return self.timeslots[pos[0]][pos[1:]]
        else:
            return self.timeslots[pos]
    # to string
    def __str__(self):
        return "\n".join(
            [
                " | ".join(
                    str(self.timeslots[t].cells[m]) for t in range(self.context.T)
                ) for m in range(self.context.M)
            ]
        )

In [176]:
# ============== Testing ==============
set_input(TestCases[1])

context = input_context()
distance = input_distances(context)

s = Schedule(context, distance)
# print entire schedule
print("\nSchedule:-")
s.randomize()

print()
print(s)
print(f"s[0][0] D value: {s[0].getCellD(0)}")
print(f"s[0][1] D value: {s[0].getCellD(1)}")
s[0][0][0] = 1
print()
print(s)
print(f"s[0][0] D value: {s[0].getCellD(0)}")
print(f"s[0][1] D value: {s[0].getCellD(1)}")
print(f"s[0].G: {s[0].G}")
print("correct")
s[0]._buildDCache()
s[0]._cache_G = None
print(f"s[0][0] D value: {s[0].getCellD(0)}")
print(f"s[0][1] D value: {s[0].getCellD(1)}")
print(f"s[0].G: {s[0].G}")

print("\n swap testing \n")
print(s)
print()
t = s[0,1]
s[0][1] = s[1][1]
s[1][1] = t
print(s)


Schedule:-

1 2 | 11 3 | 6 10
5 7 | 4 12 | 8 9
s[0][0] D value: 2.2
s[0][1] D value: 2.2

1 2 | 11 3 | 6 10
5 7 | 4 12 | 8 9
s[0][0] D value: 2.2
s[0][1] D value: 2.2
s[0].G: 3.4000000000000004
correct
s[0][0] D value: 2.2
s[0][1] D value: 2.2
s[0].G: 3.4000000000000004

 swap testing 

1 2 | 11 3 | 6 10
5 7 | 4 12 | 8 9

1 2 | 11 3 | 6 10
4 12 | 5 7 | 8 9


In [19]:
# Imports
from typing import Tuple
import random
# Schedule indexed as [time, market, k]
# Schedule[time] gives the entire time slot which is a list of T cells with each cell being a list having k shops.
class Schedule_old:
    # This is used to locate different elements of the schedule
    from dataclasses import dataclass
    @dataclass
    class Position:
        timeslot_idx: int = None
        market_idx: int = None
        shop_idx: int = None
        # validates the position object w.r.t the given context
        def validate(self, context: Context, 
                                 validate_timeslot=True, 
                                 validate_market=True, 
                                 validate_shop=True) -> bool:
            if(validate_timeslot):
                if(self.timeslot_idx < 0 
                     or self.timeslot_idx >= context.T):
                    return False
            if(validate_market):
                if(self.market_idx < 0 
                     or self.market_idx >= context.M):
                    return False
            if(validate_shop):
                if(self.shop_idx < 0 
                     or self.shop_idx >= context.K):
                    return False
            return True
            
    
    # This generates a schedule full of None values.
    def __init__(self, context: Context, distance_matrix: SymmetricMatrix):
        # save context and distance matrix
        self.context = context
        self.distance = distance_matrix
        # allocate space for schedule
        self.matrix = [None]*context.T            # T rows
        for m in range(context.T):
            self.matrix[m] = [None]*context.M       # M columns
            for t in range(context.M):
                self.matrix[m][t] = [None]*context.K  # k shops per cell
                
    def custom_sch(self, seq: list):
        #input your custom schedule. Make sure all elements are integers and no element is repeated.
        #Entering cell-wise i.e. 1 2 | 5 6
        #                        3 4 | 7 8
        i = 0
        for t in range(self.context.T):
            for m in range(self.context.M):
                for k in range(self.context.K):
                    self.matrix[t][m][k] = seq[i]
                    i = i+1
        
    
    # This randomises the schedule
    def randomize(self, seed = random.seed()):
        serial_sch = list(range(1,self.context.N+1))
        # shuffle schedule
        # random.Random(seed).shuffle(serial_sch)
        # set schedule contents to contents of serial_sch
        self.custom_sch(serial_sch)
        
    # define str funtion to print schedule as final output
    def __str__(self):
        return "\n".join(
                [
                 " | ".join(
                         " ".join(
                                 map(str, self.matrix[t][m])
                         )
                         for t in range(self.context.T)
                 ) 
                 for m in range(self.context.M)
                ]
        )
    
    # __getitem__ function overloads the [] operator
    # [x,y,z] will fetch x timeslot -> y market -> shop at position z in the cell
    # [x] will fetch the entire x timeslot
    def __getitem__(self, pos):
        # magic
        if (type(pos) == tuple):
            l = len(pos)
            if l == 1:
                self.matrix[pos[0]]
            elif l == 2:
                return self.matrix[pos[0]][pos[1]]
            elif l == 3:
                return self.matrix[pos[0]][pos[1]][pos[2]]
        else:
            return self.matrix[pos]

    # ============= Problem Specific =============
    # calculates the G value of a given schedule element or the entire schedule
    def calc_G(self, position: Position = Position()) -> float:
        if (position.timeslot_idx != None 
             and position.market_idx != None):
            # calc cell G value
            return self.calc_cell_G(position)
        elif (position.timeslot_idx != None):
            # calc timeslot G value
            return self.calc_timeslot_G(position)
        else:
            # calc schedule G value
            G = 0
            pos = self.Position(0,None,None)
            while(pos.timeslot_idx < self.context.T):
                G += self.calc_timeslot_G(pos)
                pos.timeslot_idx += 1
            return G
    
    # calculates the D value between two cells
    # this does not multiply the values with C
    def calc_intercell_D(self, pos_1: Position, pos_2: Position) -> float:
        # validate cell positions
        if (not pos_1.validate(self.context, validate_shop=False)
                or not pos_2.validate(self.context, validate_shop=False)):
            raise Exception("Invalid position:", pos_1, pos_2)
        if (pos_1.timeslot_idx != pos_2.timeslot_idx):
            raise Exception("Cannot calculate G value between cells of different timeslots.", pos_1, pos_2)
        if(pos_1 == pos_2):
            return 0
        # calculate inter-cell G value
        D = 0
        shops_1 = self.matrix[pos_1.timeslot_idx][pos_1.market_idx]
        shops_2 = self.matrix[pos_2.timeslot_idx][pos_2.market_idx]
        k = self.context.K
        for i in range(k):
            for j in range(k):
                D += self.distance[shops_1[i]-1, shops_2[j]-1]
        return D
    
    # calculates the G value of a timeslot
    def calc_timeslot_G(self, position: Position) -> float:
        # check if position valid
        if(not position.validate(self.context, validate_market=False, validate_shop=False)):
            raise Exception("Invalid position:", position)
        # calculate S and D
        S, D = 0, 0
        # position iterators
        pos_1, pos_2 = self.Position(position.timeslot_idx,0,None), self.Position(position.timeslot_idx,0,None)
        # pos_1 runs through each cell along timeslot
        while pos_1.market_idx < self.context.M:
            # add current cell G
            S += self.calc_cell_G(pos_1)
            # add intra cell G w.r.t current cell
            pos_2.market_idx = pos_1.market_idx + 1
            while pos_2.market_idx < self.context.M:
                D += self.calc_intercell_D(pos_1, pos_2)
                pos_2.market_idx += 1
            pos_1.market_idx += 1
        # G = S + C*D
        return S + self.context.C * D

    # calculates the G value of a single cell
    def calc_cell_G(self, position: Position) -> float:
        # check if position valid
        if(not position.validate(self.context, validate_shop=False)):
            raise Exception("Invalid position:", position)
        shops = self.matrix[position.timeslot_idx][position.market_idx]
        S = 0
        for i in range(len(shops)):
            for j in range(i+1, len(shops)):
                S += 1 - self.distance[shops[i]-1, shops[j]-1]
        return S
    # ============================================

# Utility functions
## input_context()
Inputs data from the user and return a `Context` object.
## input_distances(context)
Takes a `Context` object, then inputs the distance matrix from the user and returns the `SymmetricMatrix` object of the distance matrix.

In [49]:
def input_context() -> Context:
    K, M, T = [int(x) for x in [input(),input(),input()]]
    C = float(input())
    return Context(K,M,T,C)

def input_distances(context: Context) -> SymmetricMatrix:
    mat = SymmetricMatrix(size=context.N)
    for i in range(context.N):
        for idx, dist in enumerate(input().split()):
            mat[i,idx] = float(dist)
    return mat

# Testing

## TestCases

In [21]:
for t in range(len(TestCases)):
    print(f"\nTst Case {t}:-\n", TestCases[t])


Tst Case 0:-
 
                  2
                  2
                  1
                  1
                  0 0.4 0.8 1
                  0.4 0 0.6 0.7
                  0.8 0.6 0 0.3
                  1 0.7 0.3 0
                  

Tst Case 1:-
 
                  2
                  2
                  3
                  1
                  0 0.0 0.8 0.9 0.2 0.1 1.0 0.8 0.2 0.3 0.8 0.8
                  0.0 0 0.8 1.0 0.2 0.1 0.8 0.9 0.2 0.2 0.8 1.0
                  0.8 0.8 0 0.1 0.8 0.9 0.2 0.2 0.7 0.9 0.0 0.2
                  0.9 1.0 0.1 0 0.7 0.9 0.0 0.2 0.9 0.9 0.1 0.1
                  0.2 0.2 0.8 0.7 0 0.1 0.8 1.0 0.3 0.2 0.9 0.7
                  0.1 0.1 0.9 0.9 0.1 0 0.8 0.9 0.1 0.2 0.8 0.9
                  1.0 0.8 0.2 0.0 0.8 0.8 0 0.0 0.9 0.8 0.1 0.0
                  0.8 0.9 0.2 0.2 1.0 0.9 0.0 0 0.8 0.9 0.2 0.0
                  0.2 0.2 0.7 0.9 0.3 0.1 0.9 0.8 0 0.2 0.8 0.8
                  0.3 0.2 0.9 0.9 0.2 0.2 0.8 0.9 0.2 0 0.8 0.9
                  0.8 0.8

## Context

In [22]:
# ============== Testing ==============
print(Context(K=2,C=2,T=2,M=2))
# =====================================

Context(K=2, M=2, T=2, C=2, N=8)


## SymmetricMatrix

In [46]:
# ============== Testing ==============
m = SymmetricMatrix(4)
m[1,2] = 0.12
m[0,3] = 1.2
print(m)
# =====================================

SymmetricMatrix(size=4) [
	None
	None None
	None 0.12 None
	1.2 None None None
]


## Schedule

In [54]:
# ============== Testing ==============
set_input(TestCases[1])

context = input_context()
distance = input_distances(context)

s = Schedule(context, distance)
# print entire schedule
print("\nSchedule:-")
print(s)

# 2nd timeslot
print("s[0]:",s[0])

# 2nd cell in 2nd timeslot
print("s[0][1]:", s[0,1])

# 1st shop in 2nd cell in 2nd timeslot
print("s[0][1][0]:", s[0,1,0])

# print G value of 1st market in 1st timeslot
# print("\nG-value[t=0][m=0] =",s.calc_G(s.Position(0,0)))  #uncomment
# print("\nG-value[t=0][m=1] =",s.calc_G(s.Position(0,1)))  #uncomment
# print("\nG-value[t=0] =",s.calc_G(s.Position(0)))   #uncomment
# print("\nG-value =",s.calc_G())   #uncomment
# =====================================


Schedule:-
[ N o n e ,   N o n e ] | [ N o n e ,   N o n e ]
[ N o n e ,   N o n e ] | [ N o n e ,   N o n e ]
[ N o n e ,   N o n e ] | [ N o n e ,   N o n e ]


TypeError: &#39;Schedule&#39; object is not subscriptable

# Benchmarks

In [25]:
# benchmark setup
set_input(TestCases[2])
context = input_context()
distance = input_distances(context)
s = Schedule(context, distance)
s.randomize()

print(s)

1 2 3 4 5 6 7 8 9 10 | 51 52 53 54 55 56 57 58 59 60
11 12 13 14 15 16 17 18 19 20 | 61 62 63 64 65 66 67 68 69 70
21 22 23 24 25 26 27 28 29 30 | 71 72 73 74 75 76 77 78 79 80
31 32 33 34 35 36 37 38 39 40 | 81 82 83 84 85 86 87 88 89 90
41 42 43 44 45 46 47 48 49 50 | 91 92 93 94 95 96 97 98 99 100


In [26]:
%%timeit
s.calc_G()

2.31 ms ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Algo testing utilities
This code is used to test an algo's performance on different test cases in order to compare multiple different algos.


## Test DS definitions
These are used to manage the test perfomance data of an algo.

In [27]:
from dataclasses import dataclass
from typing import List, Any

@dataclass
class TestResult:
    G_values: List[float] # G values for each exexution
    Times: List[float]      # Execution time for each execution
    Iterations: List[int] # No. of iteration returned by each execution
    def to_dict(self):
        return {
            'G_value': self.G_values,
            'Time': self.Times,
            'Iterations': self.Iterations,
        }

@dataclass
class TestCase:
    context: Context
    distance: SymmetricMatrix

@dataclass
class TestParams:
    test_case_repetition: int
    generated_test_case_count: int
    test_cases: List[TestCase]

## Algo definition
This defines what an algo is supposed to be. An algo that must be tested has to be a function that takes a `Context` and an `SymmetricMatrix` as input, and returns a tuple of the output string and the best G value as output.

In [28]:
from typing import Callable, Tuple

# Define an algo
# Inputs: Context, 
#         Distance matrix
# Outputs: str of best schedule,
#          G value of best schedule, 
#          no. of iterations
Algo = Callable[[Context, SymmetricMatrix], Tuple[str, float, int]]

## Test case generation

In [29]:
# Generate context
def genContext(N: int = None) -> Context:
    # returns a list of List[int,int,int] that are factors of N
    def factorize(N):
        r = []
        for i in range(N):
            for j in range(N):
                if i*j > N:
                    break
                for k in range(N):
                    if i*j*k > N:
                        break
                    elif (i*j*k == N):
                        r.append([i,j,k])
        return r
    import random as rn
    if (N == None):
        N = rn.randint(1,100)
    C = round(rn.random(), 1)
    f = factorize(N)
    while len(f) == 0:
        N = rn.randint(1,100)
        f = factorize(N)
    l = rn.choice(factorize(N))
    rn.shuffle(l)
    K, M, T = l
    return Context(K,M,T,C)

# Generate distance matrix
def genDistMatrix(context: Context) -> SymmetricMatrix:
    import random as rn
    ret = SymmetricMatrix(context.N)
    for i in range(context.N):
        ret[i, i] = 0
        for j in range(i+1, context.N):
            ret[i, j] = round(rn.random(), 1)
    return ret

# Generate test case
def genTestCase(context: Context = None) -> TestCase:
    if(context == None):
        context = genContext()
    return TestCase(context, genDistMatrix(context))


## Testing functions
These functions test the algo and record its performance.

In [30]:
from typing import Tuple, List
from timeit import default_timer as timer
import multiprocessing as mp
import ray

# init ray
ray.init(ignore_reinit_error=True)

@ray.remote
def __bench_algo(algo, ctx, dst):
    # Benchmark algo
    start = timer()
    retVal = algo(ctx, dst)
    end = timer()
    # Benchmark end
    return (end-start, retVal,)

# Takes an algo by runnig it multiple times over a given test case
@ray.remote
def test_algo_for_set(algo: Algo, tparam: TestParams, tcase: TestCase) -> Tuple[TestCase, TestResult]:
    # ray
    res = []
    for i in range(tparam.test_case_repetition):
        res.append(
            __bench_algo.remote(algo, tcase.context, tcase.distance)
        )
    res = ray.get(res)
    # extract results
    times, retVals = zip(*res)
    S, G, Iter = zip(*retVals)
    return (tcase, TestResult(G, times, Iter),)

def testAlgo(algo: Algo, test_params: TestParams) -> List[Tuple[TestCase, TestResult]]:
    # prepare test cases
    par = []
    # add generated test cases
    for i in range(test_params.generated_test_case_count):
        par.append(
            test_algo_for_set.remote(
                algo, test_params, genTestCase()
            )
        )
    # add given test cases
    for e in test_params.test_cases:
        par.append(
            test_algo_for_set.remote(
                algo, test_params, e
            )
        )
    # ray multi process test cases
    return ray.get(par)


2020-10-02 06:17:05,789	INFO resource_spec.py:205 -- Starting Ray with 8.94 GiB memory available for workers and up to 4.48 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


## Plotting functions

In [31]:
import pandas as pd

def expectedG(test_case: TestCase) -> float:
    ctx, dst = test_case.context, test_case.distance
    # calc sum of dist
    s = 0
    for r in dst.matrix:
        s += sum(r)
    s *= 2
    # calc average distance
    avgDist = s / (ctx.N ** 2)
    # create dummy schedule to calc. expected G
    dummyDst = SymmetricMatrix(ctx.N)
    for i in range(ctx.N):
        for j in range(i+1, ctx.N):
            dummyDst[i,j] = avgDist
    # return expected G
    s = Schedule(ctx, dummyDst)
    s.randomize()
    return s.calc_G()

# converts TestResult to a dataframe with normalised G values
def testresult_to_df(test_res: Tuple[TestCase, TestResult]) -> pd.DataFrame:
    tcase, tres = test_res
    # make df
    df = pd.DataFrame(tres.to_dict())
    # normalise G to expected G, i.e. estimated average G
    df['G_value'] *= 1/expectedG(tcase)
    df = df.rename(columns={'G_value': 'normalised_G'})
    # add test case values
    df['K'] = tcase.context.K
    df['M'] = tcase.context.M
    df['T'] = tcase.context.T
    df['N'] = tcase.context.N
    df['C'] = tcase.context.C
    return df

def test_out_to_df(test_out: List[Tuple[TestCase, TestResult]]) -> pd.DataFrame:
    df = pd.DataFrame()
    for e in test_out:
        df = df.append(testresult_to_df(e), sort=False, ignore_index=True)
    return df

# creates a boxplot of M*K vs normalised G
def mk_boxplot(data: pd.DataFrame):
    # group data by test case
    data.groupby(['K','M','T','C'])

## Standard test paramaters
Used for comparing different algos with the same test cases.

In [32]:
std_test_params = [
    # Test with 10 common generated test cases
    TestParams(
        test_case_repetition = 20, 
        generated_test_case_count = 0,
        test_cases = [
            genTestCase() for i in range(10)
        ]
    )
]


# Algorithms
The main algos we create.

# Genetic Algorithm

In [44]:
from typing import Tuple
import random

class GA:
    def __init__(self, context: Context, distance: SymmetricMatrix):
        # create schedule object
        self.population = Schedule (context, distance)
        self.population.randomize()
    
    def Mutation(self, timeslot):
        #select market randomly
        mother_index = random.randint(0, self.population.context.M - 1)
        father_index = random.randint(0, self.population.context.M - 1)
        
        #in case of M<=2 mutation will result in a continuous loop. Will be ttaken care of by crossover
        if mother_index is father_index:
            return
        
        #random cell for mutation    
        motherCell = timeslot [mother_index]
        fatherCell = timeslot [father_index]    
        
        mother_min, father_min = 1, 1
        mother_k, father_k = 0,0  #just in case S is 1 for all shop pairs
        
        for i in range(len(motherCell)):
            for j in range(i+1, len(motherCell)):
                S_mother = 1 - self.population.distance[motherCell[i]-1, motherCell[j]-1]
                S_father = 1 - self.population.distance[fatherCell[i]-1, fatherCell[j]-1]
                
                if S_mother < mother_min:
                    mother_k = random.choice([i,j]) #select either element of lowest G pair
                    #mother_gene = timeslot[mother_index][mother_k]  
                    mother_min = S_mother
                    
                if S_father < father_min:
                    father_k = random.choice([i,j])
                    #father_gene = timeslot[father_index][father_k]
                    father_min = S_father
                    
        #swap shops
        temp = timeslot[mother_index][mother_k]
        timeslot[mother_index][mother_k] = timeslot[father_index][father_k]
        timeslot[father_index][father_k] = temp
        
        #print (mother_index, mother_k, mother_gene, end='\n')
        #print (father_index, father_k, father_gene, end='\n')
        
    def Evolution(self) -> Tuple[str, float, int]:
    # create schedule object
    
        #Pick a time schedule to mutate
        Time_n = random.randint(0,self.population.context.T-1)
        G_parent = self.population.calc_G(self.population.Position(timeslot_idx=Time_n))
    
        print("Parent: ",self.population[Time_n], G_parent)
        print('Parent: \n', self.population)
    
        #Offspring after mutation
        self.Mutation(self.population[Time_n])
        G_child = self.population.calc_G(s.Position(timeslot_idx=Time_n))
    
        print("Mutant: ",self.population[Time_n], G_child)
        print('Mutant: \n', self.population)
        
        return (str(self.population), G_child, 1)

## Random Search

In [34]:
from typing import Tuple
def randomSearch(context: Context, distance: SymmetricMatrix) -> Tuple[str, float, int]:
    # create schedule object
    s = Schedule(context, distance)
    # setup vars
    bestG = -1
    bestS = None
    iterations = 0
    from timeit import default_timer as timer
    # run for 2 sec
    start = timer()
    while (timer() - start < 2):
        s.randomize()
        G = s.calc_G()
        if(G > bestG):
            bestS = str(s)
            bestG = G
        iterations +=1
    return (bestS, bestG, iterations,)


### Test on pre-defined test cases

In [35]:
# ============================
# Set input for colab notebook
# ============================
set_input(TestCases[1])
# ============================

# Take input
context = input_context()
distance = input_distances(context)

# run random search
bestS, bestG, iterations = randomSearch(context, distance)

print(bestS)
print("\nG:", bestG)
print("Iterations:", iterations)

1 2 | 5 6 | 9 10
3 4 | 7 8 | 11 12

G: 15.600000000000001
Iterations: 11313


In [48]:
# ============================
# Set input for colab notebook
# ============================
set_input(TestCases[1])
# ============================

# Take input
context = input_context()
distance = input_distances(context)

# create object of GA
ga = GA(context, distance)

bestS, bestG, iterations = ga.Evolution()

print("Schedule:\n",bestS)
print("\nG:", bestG)
print("Iterations:", iterations)

Parent:  [[1, 2], [3, 4]] 5.4
Parent: 
 1 2 | 5 6 | 9 10
3 4 | 7 8 | 11 12
Mutant:  [[1, 2], [3, 4]] 5.4
Mutant: 
 1 2 | 5 6 | 9 10
3 4 | 7 8 | 11 12
Schedule:
 1 2 | 5 6 | 9 10
3 4 | 7 8 | 11 12

G: 5.4
Iterations: 1


### Test on generated test cases

In [36]:
# test algo
test_result = testAlgo(randomSearch, std_test_params[0])

2020-10-02 06:17:14,834	WARNING worker.py:1779 -- WARNING: 12 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-02 06:17:18,868	WARNING worker.py:1779 -- WARNING: 13 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-02 06:17:18,871	WARNING worker.py:1779 -- WARNING: 14 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-02 06:17:18,872	WARNING worker.py:1779 -- WARNING: 15 PYTHON workers have been started. This could be a 

In [37]:
# show mean performance for different test cases
analyse = test_out_to_df(test_result).groupby(['K','M','T','C']).mean()
analyse

normalised_G      Time  Iterations      N
K  M  T  C                                             
1  5  20 0.8      0.916828  2.001641      697.05  100.0
   14 2  0.7      1.040134  2.001121      787.25   28.0
2  2  1  0.4      0.727273  2.000628    35019.20    4.0
   13 3  0.7      1.031280  2.002386      505.40   78.0
   16 2  0.8      1.021754  2.002425      525.15   64.0
3  1  3  1.0      0.955189  2.000149    27183.90    9.0
      23 0.3      1.018061  2.000701     3886.05   69.0
5  6  2  0.2      1.027567  2.001853      971.40   60.0
23 2  1  0.9      0.986085  2.000967     1192.95   46.0
33 1  2  0.9      0.997811  2.001594     1016.35   66.0